In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a03_pabellon as pb

pd.set_option('display.max_columns', None)

# Agrega las columnas de poblacion de interes
ANIOS_INTERES = [f"{i}" for i in [2025, 2030, 2040]]

2025-07-03 22:06:59.726 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr


## Obtencion de casos por area de influencia INT

In [3]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos de HMD
casos_hmd = casos_macroproceso.query("tipo_paciente == 'hmd'")

## Obtencion de porcentajes de Hemodinamia

In [4]:
# Define la ruta donde estan los porcentajes de Hemodinamia
RUTA_PLANILLA_PORCENTAJES = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

# Solamente lee los diagnosticos y el porcentaje de Hemodinamia
datos_hmd = pd.read_excel(
    RUTA_PLANILLA_PORCENTAJES,
    usecols=["Diagnostico", "Porcentaje Pacientes Hemodinamia", "Horas en Hemodinamia"],
)

# Deja los diagnosticos que SI aportan a la cantidad de pabellones de HMD (porcentaje > 0)
datos_hmd = datos_hmd.query("`Porcentaje Pacientes Hemodinamia` > 0")

# Formatea la columna de diagnosticos
datos_hmd["Diagnostico"] = datos_hmd["Diagnostico"].str.split(" - ").str[0]

# Deja como indice a los diagnosticos
datos_hmd = datos_hmd.set_index("Diagnostico")

# Indica el tiempo quirurgico por diagnostico promedio (o 75%)
tiempos_hmd = datos_hmd["Horas en Hemodinamia"]

In [5]:
# Multiplica los casos en HMD con sus tiempos quirurgicos
horas_en_hmd_por_casos = casos_hmd[ANIOS_INTERES].mul(tiempos_hmd, axis=0)

# Obtiene la cantidad de horas laborales
horas_laborales = features.calcular_horas_laborales(
    2017, 2040, horas_por_dia_semana=12, incluir_sabado=True, horas_sabado=6
)

# Obtiene cantidad de pabellones de hemodinamia
pabellones_hmd = pb.calcular_cantidad_de_pabellones_necesarios(
    horas_en_hmd_por_casos, horas_laborales
)

# Obtiene la cantidad de pabellones totales por anio
cantidad_de_pabellones_necesarios = pabellones_hmd.sum()

Horas laborales por año calculadas:
+---+------+-------------------------------+
|   | anio | horas_laborales_con_sabado_6h |
+---+------+-------------------------------+
| 0 | 2017 |             3282              |
| 1 | 2018 |             3264              |
| 2 | 2019 |             3282              |
| 3 | 2020 |             3312              |
| 4 | 2021 |             3300              |
+---+------+-------------------------------+

Cantidad de pabellones necesarios calculada:
+-------------+------+------+------+------+------+------+------+------+-----------------------+------+------+------+------+-----------------------+------+------+------+------+------+------+------+------+------+-----------------------+
| Diagnostico | 2017 | 2018 | 2019 | 2020 | 2021 | 2022 | 2023 | 2024 |         2025          | 2026 | 2027 | 2028 | 2029 |         2030          | 2031 | 2032 | 2033 | 2034 | 2035 | 2036 | 2037 | 2038 | 2039 |         2040          |
+-------------+------+------+------+------+

In [6]:
print(f"> Pabellones de Hemodinamia 2040: {cantidad_de_pabellones_necesarios['2040']:.2f}")

> Pabellones de Hemodinamia 2040: 0.15


In [7]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = (
    pd.read_excel(
        "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
    )
    .sort_values("Diagnostico")
)
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2040
resumen_quirurgicos = pd.DataFrame(
    {
        "porcentaje_hemodinamia": datos_hmd["Porcentaje Pacientes Hemodinamia"],
        "casos_hmd_2040": casos_hmd["2040"],
        "tiempo_quirurgicos": tiempos_hmd,
        "horas_hmd_2040": horas_en_hmd_por_casos["2040"],
        "horas_laborales_2040_pabellon_12_hrs": horas_laborales["2040"],
        "cantidad_de_pabellones_hmd": pabellones_hmd["2040"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="inner", left_index=True, right_index=True
)

In [8]:
a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "casos_hmd": casos_hmd,
    "resumen_duraciones_hmd": tiempos_hmd,
    "tiempo_utilizado_pabellon": horas_en_hmd_por_casos,
    "horas_laborales_por_anio": horas_laborales,
    "pabellones_desg": pabellones_hmd,
    "pabellones_totales": cantidad_de_pabellones_necesarios,
}

In [9]:
with pd.ExcelWriter("../data/interim/2.1_estimacion_pabellones_HMD_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)